### Cleaning data on Career Field

In [3]:
# Credits to Victor for the code
# Import libraries

# basic libraries
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
# import ast
pd.set_option('display.max_rows', 500) # to show more rows.

# for translation
import sys
from Translate_data import translate_data
import warnings
warnings.filterwarnings('ignore')

import torch

In [4]:
# Load Data
df_raw = pd.read_csv('saber_combined_all_fields.csv')
df_raw = df_raw.drop(columns = 'Unnamed: 0')
df_raw.head(3)

,estu_tipodocumento,estu_nacionalidad,estu_genero,estu_fechanacimiento,estu_exterior,periodo,estu_consecutivo,estu_estadocivil,estu_estudiante,estu_pais_reside,...,punt_biologia,punt_quimica,punt_fisica,punt_ciencias_sociales,punt_filosofia,punt_ingles,desemp_ingles,profundiza,puntaje_prof,desemp_prof
0,TI,COLOMBIA,M,30/07/1996,NaN,20134,EK201340233804,Soltero,ESTUDIANTE,COLOMBIA,...,39.0,42.0,33.0,33.0,29.0,38.0,A-,PUNT_INTERDISC_MEDIOAMBIENTE,49.0,NaN
1,CC,COLOMBIA,M,13/04/1994,NaN,20133,EK201330220754,Soltero,ESTUDIANTE,COLOMBIA,...,52.0,52.0,54.0,44.0,47.0,51.0,A1,PUNT_PROFUNDIZA_LENGUAJE,6.0,II
2,CC,COLOMBIA,F,08/12/1991,NaN,20134,EK201340246502,Soltero,ESTUDIANTE,COLOMBIA,...,50.0,51.0,28.0,46.0,45.0,43.0,A-,PUNT_PROFUNDIZA_BIOLOGIA,5.0,I


In [5]:
# Translate data
df = translate_data(df_raw, 'spanish', 'english')
df.head(3)

,your_type_of_document,your_nationality,your_gender,your_birthdate,your_foreigner,period,your_consecutive,your_marital_status,your_student,your_country_resides,...,score_biology_saber_11,score_chemistry_saber_11,score_physics_saber_11,score_social_science_saber_11,score_philosophy_saber_11,score_english_saber_11,score_english_saber_11_category,optative_field_saber_11,score_optative_saber_11,optative_category_saber_11
0,TI,COLOMBIA,M,30/07/1996,NaN,20134,EK201340233804,Single,STUDENT,COLOMBIA,...,39.0,42.0,33.0,33.0,29.0,38.0,A-,SCORE_INTERDISC_ENVIRONMENT,49.0,NaN
1,CC,COLOMBIA,M,13/04/1994,NaN,20133,EK201330220754,Single,STUDENT,COLOMBIA,...,52.0,52.0,54.0,44.0,47.0,51.0,A1,SCORE_DEEPEN_LANGUAGE,6.0,II
2,CC,COLOMBIA,F,08/12/1991,NaN,20134,EK201340246502,Single,STUDENT,COLOMBIA,...,50.0,51.0,28.0,46.0,45.0,43.0,A-,SCORE_DEEPEN_BIOLOGY,5.0,I


In [6]:
# Wrangling Department

# - - - correct code - - - 
# Change code from float to int
df['your_dept_code_resides'] = df[['your_dept_code_resides']].apply(np.int64).astype(str)
df['your_dept_code_resides.1'] = df[['your_dept_code_resides.1']].apply(np.int64).astype(str)

# two-digit code for Department (Colombian States)
df['your_dept_code_resides'] = ["0"+i if len(i)==1 else i for i in df['your_dept_code_resides']]
df['your_dept_code_resides.1'] = ["0"+i if len(i)==1 else i for i in df['your_dept_code_resides.1']]

In [7]:
# Wrangling Municipality

# - - - correct name - - - 
import re
municipality_list = list(df['cole_municipality_location'].unique()) + \
                    list(df['your_municipality_resides'].unique()) + \
                    list(df['your_inst_municipality'].unique()) + \
                    list(df['your_municipality_resides.1'].unique())

municipality_list = pd.DataFrame(data=municipality_list, columns=['Name'])#.unique()
municipality_list = list(municipality_list.Name.unique())
municipality_list = [str(i) for i in municipality_list]
municipality_list = sorted(municipality_list)
municipality_values = sorted(municipality_list)

municipality_values = [re.sub(r'BOGOTÁ, D.C.', 'BOGOTÁ D.C.', mun) for mun in municipality_values]
municipality_values = [re.sub(r'CARTAGENA$', 'CARTAGENA DE INDIAS', mun) for mun in municipality_values]
municipality_values = [re.sub(r'CHIQUINQUIRA$', 'CHIQUINQUIRÁ', mun) for mun in municipality_values]
municipality_values = [re.sub(r'CIÉNEGA$', 'CIÉNAGA', mun) for mun in municipality_values]
municipality_values = [re.sub(r'PUERTO ASIS$', 'PUERTO ASÍS', mun) for mun in municipality_values]
municipality_values = [re.sub(r'POPAYAN$', 'POPAYÁN', mun) for mun in municipality_values]
municipality_values = [re.sub(r'FACATATIVÁ$', 'FACATATIVA', mun) for mun in municipality_values]

zip_iterator = zip(municipality_list, municipality_values) # Get pairs of elements
municipality_dict = dict(zip_iterator)  # Convert to dictionary

municipality_columns = ['cole_municipality_location', 'your_municipality_resides',
                        'your_inst_municipality', 'your_municipality_resides.1']

for col in municipality_columns:
    df[col] = df[col].map(municipality_dict)
    
# - - - correct code - - - 
# Change code from float to int
df['your_municipality_code_resides'] = df[['your_municipality_code_resides']].apply(np.int64).astype(str)
df['your_municipality_code_resides.1'] = df[['your_municipality_code_resides.1']].apply(np.int64).astype(str)

# two-digit code for Department (Colombian States)
df['your_municipality_code_resides'] = ["0"+i if len(i)==4 else i for i in df['your_municipality_code_resides']]
df['your_municipality_code_resides.1'] = ["0"+i if len(i)==4 else i for i in df['your_municipality_code_resides.1']]


In [8]:
# Clean data on the career field
# dictionary of categorical variables
import urllib.request as urllib2
import ast
contents = str("")
for line in urllib2.urlopen("https://raw.githubusercontent.com/vcuspinera/Datasets/main/omdena/colombia-career-recommender-system/translate_cat.txt"):
    contents += str(line)[2:-3]
contents += "}"
char_to_replace = {
    "\\r": "", "\\": "", # characters
    "xc3x81": "Á", "xc3x89": "É", "xc3x8d": "Í", "xc3x93": "Ó", "xc3x9a": "Ú", "xc3x9c": "Ü", "xc3x91": "Ñ", # capital letters
    "xc3xa1": "á", "xc3xa9": "é", "xc3xad": "í", "xc3xb3": "ó", "xc3xba": "ú", "xc3xb1": "ñ" # lowercase letters
}
for key, value in char_to_replace.items():
    contents = contents.replace(key, value)
dic_cat = ast.literal_eval(contents)

In [9]:
# Add the undergraduate core field variable
df['your_undergraduate_core_field'] = df[['your_undergraduate_core']].replace(dic_cat['your_undergraduate_core_field'])
# Change NULL values to UNCLASSIFIED label
df['your_undergraduate_core'] = df['your_undergraduate_core'].replace(np.nan, "UNCLASSIFIED")

In [10]:
df['your_undergraduate_core'].value_counts()

ADMINISTRATION                                  35790
LAW                                             15263
PUBLIC ACCOUNTING                               14334
EDUCATION                                       14219
INDUSTRIAL ENGINEERING                          10660
PSYCHOLOGY                                      10518
UNCLASSIFIED                                     8614
ENVIRONMENTAL, SANITARY ENGINEERING              8231
CIVIL ENGINEERING                                7736
SOCIAL COMMUNICATION, JOURNALISM                 7479
ECONOMY                                          7243
COMPUTER SYSTEMS, TELEMATICS ENGINEERING         6805
MECHANICAL ENGINEERING                           5382
DESIGN                                           5263
ARCHITECTURE                                     3969
SOCIOLOGY, SOCIAL WORK                           3832
NURSING                                          3440
ELECTRONIC ENGINEERING, TELECOMMUNICATIONS       2845
MEDICINE                    

In [11]:
df['your_undergraduate_core'].isnull().any()

False

In [12]:
df.shape

(212010, 144)

In [13]:
# drop the Unclassified category
df = df[df['your_undergraduate_core']!='UNCLASSIFIED']

In [14]:
df.shape

(203396, 144)

The 8614 rows containing the unclassified category have been removed. We still have plenty of data to build ou recommendation model on.

### Selecting the features to Train the model

I will select all the saber 11 scores as the input to the model. The 'your_undergraduate_core_field' will be used as the target. There is high imbalance of categories still, which will be handled later in the notebook.

In [15]:
input_features = ['score_language_saber_11', 'score_mathematics_saber_11', 'score_biology_saber_11', 'score_chemistry_saber_11',
                 'score_physics_saber_11', 'score_social_science_saber_11', 'score_philosophy_saber_11', 
                  'score_english_saber_11', 'your_undergraduate_core']
target_feature = 'your_undergraduate_core'

In [16]:
data = df[input_features]

In [17]:
data.isnull().any()

score_language_saber_11          False
score_mathematics_saber_11       False
score_biology_saber_11           False
score_chemistry_saber_11         False
score_physics_saber_11           False
score_social_science_saber_11    False
score_philosophy_saber_11        False
score_english_saber_11           False
your_undergraduate_core          False
dtype: bool

In [18]:
target_data = data['your_undergraduate_core']

In [19]:
input_data = data.drop('your_undergraduate_core', axis=1)

In [20]:
input_data.head(2)

,score_language_saber_11,score_mathematics_saber_11,score_biology_saber_11,score_chemistry_saber_11,score_physics_saber_11,score_social_science_saber_11,score_philosophy_saber_11,score_english_saber_11
0,32.0,43.0,39.0,42.0,33.0,33.0,29.0,38.0
1,48.0,64.0,52.0,52.0,54.0,44.0,47.0,51.0


### Preprocessing the Data for the Pytorch Model

In [21]:
# Split the data into train, val and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_data, target_data, test_size = 0.05)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.08)

In [22]:
print(X_train.shape, y_train.shape)

(177767, 8) (177767,)


In [23]:
print(X_val.shape, y_val.shape)

(15459, 8) (15459,)


In [24]:
print(X_test.shape, y_test.shape)

(10170, 8) (10170,)


In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [26]:
# Function to preprocess the input features
def scale_input(X, scaler):
    X_scaled = scaler.transform(X)
    return X_scaled

In [27]:
X_train_s = scale_input(X_train, scaler)

In [28]:
X_train_s[1]

array([-0.87003763, -0.6357859 , -0.55771466, -0.88359363, -0.8941784 ,
       -0.78527545,  0.29711551, -0.69545886])

In [29]:
X_test_s = scale_input(X_test, scaler)
X_val_s = scale_input(X_val, scaler)

In [30]:
X_train_s.shape

(177767, 8)

In [31]:
type(X_train_s)

numpy.ndarray

In [32]:
# Now we can encode the labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [33]:
le.fit(y_train)

LabelEncoder()

In [34]:
y_train_e = le.transform(y_train)
y_val_e = le.transform(y_val)
y_test_e = le.transform(y_test)

In [35]:
y_test_e

array([46, 49, 46, ...,  0, 19,  0])

In [36]:
y_train = torch.tensor(y_train_e, dtype=torch.long)
y_train

tensor([22, 17,  6,  ...,  0, 47, 35])

In [37]:
y_val = torch.tensor(y_val_e, dtype=torch.long)
y_test = torch.tensor(y_test_e, dtype=torch.long)

In [38]:
y_test[:20]

tensor([46, 49, 46,  6, 18,  2, 29, 19, 14, 49,  0, 19,  0, 39, 29, 26, 26, 29,
         0,  0])

In [39]:
print(len(list(le.classes_)))
list(le.classes_)

57


['ADMINISTRATION',
 'ADMINISTRATIVE ENGINEERING',
 'ADVERTISING',
 'AGRICULTURAL, FORESTRY ENGINEERING',
 'AGROINDUSTRIAL AND FOOD ENGINEERING',
 'AGRONOMIC AND LIVESTOCK ENGINEERING',
 'AGRONOMY',
 'ANTHROPOLOGY, LIBERAL ARTS',
 'ARCHITECTURE',
 'BACTERIOLOGY',
 'BIOLOGY, MICROBIOLOGY',
 'BIOMEDICAL ENGINEERING',
 'CHEMICAL ENGINEERING',
 'CHEMISTRY',
 'CIVIL ENGINEERING',
 'COMPUTER SYSTEMS, TELEMATICS ENGINEERING',
 'DENTISTRY',
 'DESIGN',
 'ECONOMY',
 'EDUCATION',
 'ELECTRICAL ENGINEERING',
 'ELECTRONIC ENGINEERING, TELECOMMUNICATIONS',
 'ENVIRONMENTAL, SANITARY ENGINEERING',
 'GEOGRAPHY, HISTORY',
 'GEOLOGY, OTHER NATURAL SCIENCE PROGRAMS',
 'INDUSTRIAL ENGINEERING',
 'LAW',
 'LIBRARY, OTHERS OF SOCIAL AND HUMAN SCIENCES',
 'MATH, STATISTICS',
 'MECHANICAL ENGINEERING',
 'MEDICINE',
 'MILITARY OR POLICE TRAINING',
 'MINING, METALLURGY ENGINEERING',
 'MODERN LANGUAGES, LITERATURE, LINGUISTICS',
 'MUSIC',
 'NURSING',
 'NUTRITION AND DIET',
 'ODONTOLOGY',
 'OPTOMETRY, OTHER HEALTH SC

In [40]:
type(y_test)

torch.Tensor

In [41]:
# covert the input arrays to tensor
# First convert to float
X_train_s = X_train_s.astype(np.float32)
X_val_s = X_val_s.astype(np.float32)
X_test_s = X_test_s.astype(np.float32)

X_train_t = torch.tensor(X_train_s)
X_val_t = torch.tensor(X_val_s)
X_test_t = torch.tensor(X_test_s)

In [42]:
# Setting up Torch Dataloaders
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

train_dataset = TensorDataset(X_train_t, y_train)
test_dataset = TensorDataset(X_test_t, y_test)
val_dataset = TensorDataset(X_val_t, y_val)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4, pin_memory=True)

dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
dataset_sizes

{'train': 177767, 'val': 15459}

In [43]:
train_loader

In [44]:
# Lets have a look at the first batch of the train_loader
for x, y in train_loader:
    print(x[0])
    print(x.shape)
    print(y)
    print(y.shape)
    break

tensor([-0.2090, -0.4763, -0.0700, -1.1218,  0.4015, -0.4404, -0.2393, -0.4210])
torch.Size([32, 8])
tensor([35, 25,  0, 17, 46, 18,  0,  0, 14,  0, 31, 18, 50, 17,  0, 45, 49, 13,
        14,  0, 25,  0, 25, 13, 14, 26, 44, 20, 49, 28, 26, 22])
torch.Size([32])


In [45]:
# Lets have a look at the first batch of the val_loader
for x, y in val_loader:
    print(x[0])
    print(x.shape)
    print(y)
    print(y.shape)
    break

tensor([ 0.7165,  0.4806,  1.0274,  0.4263, -0.5952,  0.0194,  0.5117,  0.4711])
torch.Size([32, 8])
tensor([22, 40, 29,  0, 45, 26, 46, 22, 46, 17, 26, 26, 40, 29, 35,  0, 17, 20,
        31,  0, 46, 26, 26, 29, 22, 25, 14,  0, 44, 29, 35, 18])
torch.Size([32])


In [46]:
# Lets have a look at the first batch of the test_loader
for x, y in test_loader:
    print(x[0])
    print(x.shape)
    print(y)
    print(y.shape)
    break

tensor([ 0.4521, -0.1573,  0.2958,  0.6645,  0.5012,  0.2493,  1.0481, -0.6268])
torch.Size([32, 8])
tensor([46, 49, 46,  6, 18,  2, 29, 19, 14, 49,  0, 19,  0, 39, 29, 26, 26, 29,
         0,  0, 14, 45, 18,  0, 30, 49, 25, 14,  0, 32, 46,  0])
torch.Size([32])


In [47]:
dataloaders = {'train':train_loader, 'val':val_loader}
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f61c0eff790>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f61c0eff700>}

### Building the Model

In [48]:
import torch.nn.functional as F
import torch.nn as nn

In [49]:
class RecommendationModel(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.1):
        super().__init__()
        self.recommender = nn.Sequential(
            nn.Linear(in_size, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, out_size),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        out = self.recommender(x)
        return out

In [50]:
insize = 8
outsize = 57
model = RecommendationModel(insize, outsize)

In [51]:
model

RecommendationModel(
  (recommender): Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=32, out_features=64, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=64, out_features=128, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=128, out_features=256, bias=True)
    (10): ReLU()
    (11): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=256, out_features=512, bias=True)
    (13): ReLU()
    (14): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Linear(in_features=512, out_features=256, bias=True)
    (16): ReLU()
    (17): BatchNorm1d(256, eps=1e

In [52]:
for t in model.parameters():
    print(t.shape)

torch.Size([32, 8])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([64, 32])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([128, 64])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([256, 128])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([512, 256])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([256, 512])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([128, 256])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([64, 128])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([57, 64])
torch.Size([57])


In [53]:
# Try and generate some outputs using the model
for x, y in train_loader:
    out = model(x)
    break

In [54]:
out[0]

tensor([0.0026, 0.0086, 0.0072, 0.0389, 0.0407, 0.0082, 0.0149, 0.0440, 0.0835,
        0.0115, 0.0131, 0.0124, 0.0268, 0.0281, 0.0189, 0.0138, 0.0116, 0.0126,
        0.0113, 0.0141, 0.0134, 0.0117, 0.0165, 0.0286, 0.0054, 0.0219, 0.0150,
        0.0227, 0.0104, 0.0105, 0.0122, 0.0175, 0.0110, 0.0236, 0.0059, 0.0060,
        0.0067, 0.0068, 0.0251, 0.0117, 0.0081, 0.0163, 0.0160, 0.0135, 0.0280,
        0.0209, 0.0143, 0.0297, 0.0073, 0.0109, 0.0133, 0.0170, 0.0259, 0.0213,
        0.0240, 0.0236, 0.0042], grad_fn=<SelectBackward>)

In [55]:
# Change the device to cuda if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

### Setting up the Loss Function

Focal loss,originally developed for handling extreme foreground-background class imbalance in object detection algorithms, could be used as an alternative for cross-entropy loss when you have imbalanced datasets.
original paper: https://arxiv.org/abs/1708.02002

In [56]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__(weight, reduction=reduction)
        self.gamma = gamma
        self.weight = weight
        
    def forward(self, inp, target):
        ce_loss = F.cross_entropy(inp, target, reduction=self.reduction, weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1-pt)**self.gamma * ce_loss).mean()
        return focal_loss

In [57]:
fc_loss = FocalLoss()
fc_loss

FocalLoss()

In [58]:
# Try and generate some outputs and test the loss fn using the model
for x, y in train_loader:
    out = model(x)
    loss = fc_loss(out, y)
    break

In [59]:
y

tensor([14, 40, 25,  0, 29, 19,  0,  0, 22, 45, 33, 46, 25, 46,  0,  0, 45, 31,
        35, 23,  2, 19,  0, 19, 18,  0,  0, 25, 49,  0, 45,  0])

In [60]:
_, preds = torch.max(out, 1)
preds

tensor([14, 18, 22, 50, 47,  1, 43, 56, 31, 39,  8, 44, 12, 52, 51,  7, 46,  8,
        27, 50, 19,  4, 54, 44, 47,  3, 32, 16, 53, 52, 31, 29])

In [61]:
loss

tensor(3.9023, grad_fn=<MeanBackward0>)

In [62]:
# Lets write the funtion for the training loop
import time
import copy
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Compile the Model

In [63]:
import torch.optim as optim
model = model.to(device)
criterion = FocalLoss(gamma=1)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### Train the model

In [64]:
model_final = train_model(model, criterion, optimizer, num_epochs=50)

Epoch 0/49
----------
train Loss: 3.8290 Acc: 0.1681
val Loss: 3.8130 Acc: 0.1805

Epoch 1/49
----------
train Loss: 3.8197 Acc: 0.1741
val Loss: 3.8121 Acc: 0.1811

Epoch 2/49
----------
train Loss: 3.8172 Acc: 0.1764
val Loss: 3.8134 Acc: 0.1800

Epoch 3/49
----------
train Loss: 3.8170 Acc: 0.1766
val Loss: 3.8129 Acc: 0.1805

Epoch 4/49
----------
train Loss: 3.8169 Acc: 0.1766
val Loss: 3.8128 Acc: 0.1804

Epoch 5/49
----------
train Loss: 3.8167 Acc: 0.1768
val Loss: 3.8118 Acc: 0.1813

Epoch 6/49
----------
train Loss: 3.8160 Acc: 0.1775
val Loss: 3.8119 Acc: 0.1813

Epoch 7/49
----------
train Loss: 3.8162 Acc: 0.1772
val Loss: 3.8126 Acc: 0.1806

Epoch 8/49
----------
train Loss: 3.8160 Acc: 0.1774
val Loss: 3.8133 Acc: 0.1800

Epoch 9/49
----------
train Loss: 3.8187 Acc: 0.1750
val Loss: 3.8140 Acc: 0.1794

Epoch 10/49
----------
train Loss: 3.8163 Acc: 0.1771
val Loss: 3.8134 Acc: 0.1799

Epoch 11/49
----------
train Loss: 3.8162 Acc: 0.1773
val Loss: 3.8123 Acc: 0.1809

Ep

In [ ]:
model_final.save('./model_final.')